<a href="https://colab.research.google.com/github/Zain-mahfoud94/Python-Uni/blob/main/02_Probability_Distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probability Distributions

**Density Estimation**:
Modeling the probability distribution $p(\mathbf{x})$ of a random variable $\mathbf{x}$, given a finite set $\mathbf{x}_1, \dots, \mathbf{x}_N$ of observations.
For this task of density estimation, we assume that these data points are independently and identically distributed (i. i. d.).

### Table of Contents
1. [Setup](#setup)
2. [Binary Variables](#binary)
3. [Mulitnomial Variables (Optional)](#multinomial)
4. [Gaussian Distribution](#gaussian)
5. [Nonparametric Methods](#kde)

## Setup <a class="anchor" id="setup"></a>

In [ ]:
!pip install ipywidgets

In [ ]:
import numpy as np
import pylab as plt
from ipywidgets import interact

def get_data(n_samples=20):
    np.random.seed(0)
    return (np.random.rand(n_samples) > 0.5).astype(int)

def get_data_kde():
    X = np.r_[np.random.normal(-3, 1, 50), np.random.normal(3, 1, 50)]
    return X

## Binary Variables <a class="anchor" id="binary"></a>
### Bernoulli Distribution
- Implement a class representing the `BernoulliDistribution`. It should contain the following methods:
  - `__init__(self, mu)` - Initialization method that verifies that $0 \leq \mu \leq 1$.
  - `prob(self, x)` - Should return the probability of a binary sample $x \in \{0, 1\}$: $$\operatorname{Bern}(x \mid \mu)=\mu^{x}(1-\mu)^{1-x}.$$
  - `log_prob(self, x)` - Should return the log-probability of a binary sample: $$\ln \operatorname{Bern}(x \mid \mu)= \ln \left(\mu^{x}(1-\mu)^{1-x} \right)$$
  This equation should be simplified.

In [ ]:
# Bernoulli distribution
class BernoulliDistribution:
    def __init__(self, mu):
        assert 0 <= mu <= 1, 'The parameter has to be between 0 and 1.'
        self.mu = mu

    def prob(self, x):
        assert x in {0, 1}, 'The input has to be binary.'
        return self.mu**x * (1-self.mu)**(1-x)

    def log_prob(self, x):
        assert x in {0, 1}, 'The input has to be binary.'
        # Due to the log, we have to ensure that mu wont be 0 or 1.
        # In such cases, we will adjust its value by a tiny offset using `np.clip`.
        mu = np.clip(self.mu, 1e-5, 1-1e-5)
        return x * np.log(mu) + (1 - x)* np.log(1 - mu)

bern = BernoulliDistribution(mu=0.2)
bern.prob(0), bern.log_prob(0)  # Should return (0.8, -0.2231435513142097)

Consider the following observations:

In [ ]:
X = get_data()

On the assumption that the observations are drawn independently, we can construct the likelihood function
$$p(\mathcal{D} \mid \mu)=\prod_{n=1}^{N} p\left(x_{n} \mid \mu\right)=\prod_{n=1}^{N} \mu^{x_{n}}(1-\mu)^{1-x_{n}}.$$
- Implement a function `likelihood` defining the equation above.

In [ ]:
# Specify a Likelihood based on the data
def likelihood(mu, X):
    bern = BernoulliDistribution(mu)
    return np.prod([bern.prob(x) for x in X])
likelihood(.5, [1, 1, 1])  # Should return 0.125

The resulting likelihood may show a tiny values, which may lead to numerical problems during the optimization (e.g., when we have big number of samples). To ensure numerical stability as well as ease further calculation, we will make use of the logarithm as it does not change the optimum of a function.
- Implement a function `log_likelihood` returning the log likelihood.

In [ ]:
# Specify a Likelihood based on the data
def log_likelihood(mu, X):
    bern = BernoulliDistribution(mu)
    return np.sum([bern.log_prob(x) for x in X])
log_likelihood(0.5, [1, 1, 1])  # Should return -2.0794415416798357

- Plot the two different likelihood functions for different values of $0 \leq \mu \leq 1$.

In [ ]:
axis = np.linspace(0, 1, 201)

plt.subplot(121)
plt.title('Likelihood')
axis_likelihood = [likelihood(mu, X) for mu in axis]
plt.plot(axis, axis_likelihood)

# Verify that their max value are the same
idx_max = np.argmax(axis_likelihood)
maximum = axis[idx_max]
plt.vlines(maximum, *plt.ylim(), label='Max = {:.2f}'.format(maximum))
plt.legend()

plt.subplot(122)
plt.title('Log Likelihood')
axis_log_likelihood = [log_likelihood(mu, X) for mu in axis]
plt.plot(axis, axis_log_likelihood)

# Verify that their max value are the same
idx_max = np.argmax(axis_log_likelihood)
maximum = axis[idx_max]
plt.vlines(maximum, *plt.ylim(), label='Max = {:.2f}'.format(maximum))
plt.legend()

plt.show()

If we set the derivative of $\ln p(\mathcal{D} | \mu)$ with respect to $\mu$ equal to zero, we obtain the maximum likelihood estimator $$\mu_{\mathrm{ML}}=\frac{1}{N} \sum_{n=1}^{N} x_{n}.$$
- Compute the maximum likelihood estimator using our observations and verify visually that it corresponds to the maximum in both plots.

In [ ]:
# ML Solution
mu_ml = np.sum(X) / len(X)
mu_ml

### Binomial Distribution
- Implement the class `BinomialDistribution` containing the following methods:
  - `__init__(self, n, mu)` - Initialization method saving parameters and verifying their domain.
  - `prob(self, x)` - Should return the probability:  $$\operatorname{Bin}(m \mid N, \mu)=\left(\begin{array}{l} N \\ m \end{array}\right) \mu^{m}(1-\mu)^{N-m}.$$
  The normalization coefficient $$ \left(\begin{array}{l} N \\ m \end{array}\right) \equiv \frac{N !}{(N-m) ! m !}$$ can be implemented using `np.math.factorial`.

In [ ]:
class BinomialDistribution:
    def __init__(self, n, mu):
        assert 0 <= mu <= 1, 'The parameter has to be between 0 and 1.'
        self.n = n
        self.mu = mu
        self.factorial = np.math.factorial

    def prob(self, x):
        assert 0 <= x <= 10, 'The input should be between 0 and {}.'.format(self.n)
        norm_coefficient = self.factorial(self.n) / (self.factorial(self.n - x)*self.factorial(x))
        unnorm_prob = self.mu**x * (1-self.mu)**(self.n - x)
        return norm_coefficient * unnorm_prob

binom = BinomialDistribution(10 , .5)
binom.prob(5)  # Should return 0.24609375

- Plot the binomial distribution ($N=10, \mu=.25$) by using a bar plot.

In [ ]:
binom = BinomialDistribution(10 , .25)
inps = np.arange(0, 11)
probs = [binom.prob(x) for x in inps]
plt.bar(inps, probs)
plt.xlabel('x')
plt.ylabel('probability')
plt.show()

### Beta Distribution
- Implement the class `BetaDistribution` containing the following methods:
  - `__init__(self, a, b)` - Initialization method saving parameters and verifying their domain.
  - `prob(self, x)` - Should return the probability: $$\operatorname{Beta}(\mu \mid a, b)=\frac{\Gamma(a+b)}{\Gamma(a) \Gamma(b)} \mu^{a-1}(1-\mu)^{b-1}$$
  The normalization coefficient $$ \frac{\Gamma(a+b)}{\Gamma(a) \Gamma(b)}$$ can be implemented using `np.math.gamma`.

In [ ]:
class BetaDistribution:
    def __init__(self, a, b):
        assert a > 0 and b > 0, 'Both paramters have to be greater than 0.'
        self.a = a
        self.b = b
        self.gamma = np.math.gamma

    def prob(self, x):
        assert 0 < x < 1, 'Input has to be between 0 and 1.'
        norm_coefficent = self.gamma(self.a + self.b) / (self.gamma(self.a)*self.gamma(self.b))
        unnorm_prob = x**(self.a - 1) * (1 - x)**(self.b-1)
        return norm_coefficent*unnorm_prob


beta = BetaDistribution(1, 3)
beta.prob(.5)  # Should return 0.75

- Plot the distribution for the given parameter settings:
  - $a=0.1\quad b=0.1$
  - $a=1\quad b=1$
  - $a=2\quad b=3$
  - $a=8\quad b=4$

In [ ]:
axis = np.linspace(1e-2, 1-1e-2, 201)

plt.subplot(221)
plt.title('a = {}, b = {}'.format(0.1, 0.1))
beta = BetaDistribution(.1, .1)
plt.plot(axis, [beta.prob(x) for x in axis])

plt.subplot(222)
plt.title('a = {}, b = {}'.format(1, 1))
beta = BetaDistribution(1, 1)
plt.plot(axis, [beta.prob(x) for x in axis])

plt.subplot(223)
plt.title('a = {}, b = {}'.format(2, 3))
beta = BetaDistribution(2, 3)
plt.plot(axis, [beta.prob(x) for x in axis])

plt.subplot(224)
plt.title('a = {}, b = {}'.format(8, 4))
beta = BetaDistribution(8, 4)
plt.plot(axis, [beta.prob(x) for x in axis])
plt.tight_layout()
plt.show()

### Bayesian Approach for Parameter Estimation
In a Bayesian approach, the posterior distribution of $\mu$ is now determined by multiplying the prior distribution $\operatorname{Beta}(\mu \mid a, b)$ with the likelihood function $\operatorname{Bin}(m \mid N, \mu)$ and subsequent normalization.

The observations below encode a coin toss ($0$ = tails, $1$ = heads):

In [ ]:
X = [1, 1, 1]

- Define a meaningful prior distribution
- Define the likelihood function
- Define the posterior distribution

In [ ]:
prior_dist = BetaDistribution(2, 2)

def likelihood_func(mu, X) :
    return np.prod([BernoulliDistribution(mu).prob(x) for x in X])

posterior_distribution = BetaDistribution(
    a=prior_dist.a + sum(X),
    b=prior_dist.b + len(X)-sum(X),
)

In [ ]:
plt.figure(figsize=(10, 5))
axis = np.linspace(1e-2, 1-1e-2, 201)

plt.subplot(131)
plt.title('Prior Distribution')
plt.plot(axis, [prior_dist.prob(mu) for mu in axis])

plt.subplot(132)
plt.title('Likelihood Function')
plt.plot(axis, [likelihood_func(mu, X) for mu in axis])

plt.subplot(133)
plt.title('Posterior Distribution')
plt.plot(axis, [posterior_distribution.prob(mu) for mu in axis])

plt.tight_layout()
plt.show()

If our goal is to predict, as best we can, the outcome of the next trial, then we must evaluate the predictive distribution of $x$, given the observed data set $\mathcal{D}$. From the sum and product rules of probability, follows
$$p(x=1 \mid \mathcal{D})=\int_{0}^{1} p(x=1 \mid \mu) p(\mu \mid \mathcal{D}) \mathrm{d} \mu=\int_{0}^{1} \mu p(\mu \mid \mathcal{D}) \mathrm{d} \mu=\mathbb{E}[\mu \mid \mathcal{D}]$$

That is, the optimal prediction for $1$ is given by the mean of the beta distribution:
$$p(x=1 \mid \mathcal{D})=\frac{m+a}{m+a+l+b}$$
- Implement a `predict` function that uses the parameters of the posterior distribution to make predictions.

In [ ]:
def predict(x, posterior_distribution):
    assert x in {0, 1}, 'Input x is not binary.'
    a, b = posterior_distribution.a, posterior_distribution.b
    prob = a / (a+b)
    if x != 1:
        prob = 1 - prob
    return prob
predict(1, posterior_distribution)

In [ ]:
# Example that the posterior distribution will be close to the likelihood
# if the amount of samples is big enough.
from ipywidgets import interact
def get_data(n_samples):
    np.random.seed(100)
    return (np.random.rand(n_samples) > .8).astype(int)

@interact(n_samples=(1, 100, 1))
def tmp(n_samples=1):
    X = get_data(n_samples)
    prior_dist = BetaDistribution(5, 2)
    def likelihood_func(mu, X) :
        return np.prod([BernoulliDistribution(mu).prob(x) for x in X])
    posterior_distribution = BetaDistribution(
        a=prior_dist.a + sum(X),
        b=prior_dist.b + len(X)-sum(X),
    )
    plt.figure(figsize=(10, 5))
    axis = np.linspace(1e-2, 1-1e-2, 201)

    plt.subplot(131)
    plt.title('Prior Distribution')
    plt.plot(axis, [prior_dist.prob(mu) for mu in axis])

    plt.subplot(132)
    plt.title('Likelihood Function')
    plt.plot(axis, [likelihood_func(mu, X) for mu in axis])

    plt.subplot(133)
    plt.title('Posterior Distribution')
    plt.plot(axis, [posterior_distribution.prob(mu) for mu in axis])

    plt.tight_layout()
    plt.show()

## Multinomial Variables (Optional) <a class="anchor" id="multinomial"></a>
* Implement the following distributions:

### Categorical Distribution
$$p(\mathbf{x} \mid \boldsymbol{\mu})=\prod_{k=1}^{K} \mu_{k}^{x_{k}}$$

In [ ]:
# Generalization Bernoulli
class Categorical:
    def __init__(self, mu):
        self.mu = np.array(mu)
        assert np.all(self.mu >= 0), 'All elements have to be greater or equal 0.'
        assert np.sum(self.mu) == 1, 'The parameter has to sum to 1.'

    def prob(self, x):
        assert sum(x) == 1 and 1 in x, 'Input has to be 1-of-K encoded.'
        assert len(x) == len(self.mu), 'Input and parameter vector have to be the same length.'
        return np.prod([mu_k**x_k for x_k, mu_k in zip(x, self.mu)])

cat_dist = Categorical(mu=[.5, .5, 0])
cat_dist.prob([0, 1, 0])

### Multinomial Distribution
$$
\operatorname{Mult}\left(m_{1}, m_{2}, \ldots, m_{K} \mid \boldsymbol{\mu}, N\right)=\left(\begin{array}{c}
N \\
m_{1}, m_{2}, \ldots, m_{k}
\end{array}\right) \prod_{k=1}^{K} \mu_{k}^{m_{k}}
$$

In [ ]:
class Multinomial:
    def __init__(self, mu, n):
        self.mu = np.array(mu)
        self.n = np.array(n)
        self.factorial = np.math.factorial
        assert np.all(self.mu >= 0), 'All elements of mu have to be greater or equal 0.'
        assert np.sum(self.mu) == 1, 'The parameter mu has to sum to 1.'
        assert self.n > 0, 'Number of trials has to be greater than 0.'

    def prob(self, x):
        assert sum(x) == self.n
        norm_coef = self.factorial(self.n) / np.prod([self.factorial(x_k) for x_k in x])
        unnorm_prob = np.prod([mu_k**x_k for x_k, mu_k in zip(x, self.mu)])
        return norm_coef*unnorm_prob

mult = Multinomial([.2, .3, .5], 6)
mult.prob([1, 2, 3])  #  should be 0.13499999999999998

### Dirichlet Distribution
$$\operatorname{Dir}(\boldsymbol{\mu} \mid \boldsymbol{\alpha})=\frac{\Gamma\left(\alpha_{0}\right)}{\Gamma\left(\alpha_{1}\right) \cdots \Gamma\left(\alpha_{K}\right)} \prod_{k=1}^{K} \mu_{k}^{\alpha_{k}-1}$$
where $\alpha_{0}=\sum_{k=1}^{K} \alpha_{k}$

In [ ]:
class Dirichlet:
    def __init__(self, alpha):
        self.alpha = np.array(alpha)
        assert all(self.alpha > 0), 'All elements of alpha have to be positive.'

        # Compute the coefficient for normalization
        gamma = np.math.gamma
        alpha0 = sum(self.alpha)
        self.norm_coef = gamma(alpha0) / np.prod([gamma(a) for a in alpha])

    def prob(self, x):
        x = np.array(x)
        assert all(0 < x) and all(x < 1) and sum(x) == 1, 'Input has to be a probability vector.'
        unnorm_prob = np.prod([mu_k**(a_k-1) for mu_k, a_k in zip(x, self.alpha)])
        return self.norm_coef * unnorm_prob

dirichlet = Dirichlet([.5, .5, .5])
dirichlet.prob([.1, .1, .8]) # should be 1.7794063585429432

## Gaussian Distribution <a class="anchor" id="gaussian"></a>
* Implement the following distributions:
$$\mathcal{N}\left(x \mid \mu, \sigma^{2}\right)=\frac{1}{\left(2 \pi \sigma^{2}\right)^{1 / 2}} \exp \left\{-\frac{1}{2 \sigma^{2}}(x-\mu)^{2}\right\}$$

In [ ]:
class Gaussian:
    def __init__(self, mu, sigma):
        self.mu = np.array(mu)
        self.sigma = np.array(sigma)
        assert self.sigma > 0, 'The standard deviation has to be greater than 0.'

        # Computation of the normalization coefficient
        self.norm_coef = 1 / np.sqrt(2*np.pi*self.sigma**2)

    def prob(self, x):
        unnorm_prob = np.exp(- (x - self.mu)**2 / (2*self.sigma**2))
        return self.norm_coef * unnorm_prob

gaussian = Gaussian(0, 1)
gaussian.prob(0)  # should be 0.3989422804014327

$$\mathcal{N}(\mathbf{x} \mid \boldsymbol{\mu}, \mathbf{\Sigma})=\frac{1}{(2 \pi)^{D / 2}} \frac{1}{|\mathbf{\Sigma}|^{1 / 2}} \exp \left\{-\frac{1}{2}(\mathbf{x}-\boldsymbol{\mu})^{\mathrm{T}} \boldsymbol{\Sigma}^{-1}(\mathbf{x}-\boldsymbol{\mu})\right\}$$

In [ ]:
class MultivariateGaussian:
    def __init__(self, mu, cov):
        self.mu = np.array(mu)
        self.cov = np.array(cov)

        # For the Gaussian distribution to be normalized, all eigenvalues of the
        # covariance matrix must be strictly positive.
        # A matrix with strictly positive eigenvalues is called positive definite.
        eig_vals = np.linalg.eigvals(self.cov)
        assert np.all(eig_vals > 0), 'Matrix is not positive definite.'

        self.cov_inv = np.linalg.inv(cov)
        self.D = len(self.mu)
        self.norm_coef = (1 / (2*np.pi)**(self.D/2)) * (1 / np.prod(np.sqrt(eig_vals)))

    def prob(self, x):
        v = np.subtract(x, self.mu)
        unnorm_prob = np.exp(- (v @ self.cov_inv @ v) / 2)
        return self.norm_coef * unnorm_prob

multivariate_gaussian= MultivariateGaussian([0, 0], [[1, 0], [0, 1]])
multivariate_gaussian.prob([0, 0])  # should be 0.15915494309189535

## Nonparametric Methods <a class="anchor" id="kde"></a>
Observations are being drawn from some unknown probability density $p(\mathbf{x})$ in some $D$-dimensional space, which we shall assume to be Euclidean.
Goal: Estimate the value of $p(\mathbf{x})$.

### Kernel Density Estimators
As shown in the lecture, we can estimate the density by $$p(\mathbf{x})=\frac{K}{N V}$$ where
- $N$ is the number of observations,
- $K$ is the number of observations that lie inside some region $\mathcal{R}$, and
- $V$ is the volume of region $\mathcal{R}$.

By fixing $V$ and determining $K$ from the data, we obtain the *kernel density estimator*.

For the kernel density method, we assume that the region $\mathcal{R}$ is a small hypercube that has its center at the point $\mathbf{x}$ for which the density is to be determined.

In order to count the number $K$ of points falling within this region, it is convenient to define the following function:

$$k(\mathbf{u})=\left\{\begin{array}{ll}
1, & \left|u_{i}\right| \leq 1 / 2, \quad i=1, \ldots, D \\
0, & \text { otherwise }
\end{array}\right.$$
which represents a unit cube centered on the origin.

For the given data set, the total number of data points lying inside this cube will therefore be

$$K=\sum_{n=1}^{N} k\left(\frac{\mathbf{x}-\mathbf{x}_{n}}{h}\right)$$

When substituted into the estimation formula $p(x) =K/(NV)$, the following result is obtained for estimating the density at point $\mathbf{x}$:

$$p(\mathbf{x})=\frac{1}{N} \sum_{n=1}^{N} \frac{1}{h^{D}} k\left(\frac{\mathbf{x}-\mathbf{x}_{n}}{h}\right)$$
where we have used $V=h^D$ for the volume of a hypercube of side $h$ in $D$ dimensions.

- Implement the kernel density estimator we described above for the following observations.

In [ ]:
X = get_data_kde()

In [ ]:
class KernelDensityEstimator:
    def __init__(self, data, h):
        self.X = np.array(data)
        self.h = np.array(h)
        self.D = 1  # Here only for the 1d data set

        assert self.h > 0, 'The length of the cube needs to be greater than 0.'

    def prob(self, x):
        x = np.array(x)
        N = len(self.X)
        V = self.h**self.D
        # Count the number of samples falling inside the unit cube
        K = np.sum(self.k((x - self.X) / self.h))

        p = K / (V * N)
        return p

    def k(self, u):
        norm = np.abs(u)
        return (norm <= .5).astype(int)

- Plot the data using a histogram and plot the density using your implemented kernel density estimator.

In [ ]:
@interact(h=(0.1, 10, 0.1))
def plot(h=1):
    kde = KernelDensityEstimator(data=X, h=h)
    axis = np.linspace(-8, 8, 201)
    plt.plot(axis, [kde.prob(x) for x in axis], linewidth=3, zorder=4, color='blue')
    plt.hist(X, zorder=3, density=True)
    plt.show()